In [4]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None

from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, TimeSeriesOutcome, IntegerParameter, RealParameter)

from dike_model_function_V2_0 import (DikeNetwork,DikeNetworkTS)  # @UnresolvedImport
from problem_formulation_V2_0 import get_model_for_actor_problem_formulation

In [5]:
# Problem formulations:
#    get_model_for_actor_problem_formulation(problem_formulation_id, outcome_type='time_series')
#    problem_formulation_id options
#        1 - RWS (GOOD)
#        2 - Environmental interest group
#        3 - Transport company 
#        4 - Delta commission
#        5 - Gelderland (GOOD)
#        6 - Overijssel (GOOD)s
#        7 - Dike rings 1 and 2
#        8 - Dike ring 3
#        9 - Dike ring 4
#        10 - Dike ring 5
#
#    outcome_type options
#        'time_series' (default)
#        'scalar'


model, planning_steps = get_model_for_actor_problem_formulation(1,outcome_type='scalar')

In [ ]:
#for lever in model.levers:
#    print(lever.name)

In [ ]:
#for unc in model.uncertainties:
#    print(repr(unc))

In [ ]:
#for outcome in model.outcomes:
#    print(repr(outcome))

In [ ]:
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)
 
with MultiprocessingEvaluator(model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=15000, policies=1)

In [ ]:
results = experiments, outcomes

In [ ]:
import ema_workbench

In [ ]:
#ema_workbench.util.utilities.save_results(results, 'results15000.tar.gz')

In [ ]:
experiments.iloc[1:2]

In [ ]:
#cleaned_experiments = 
experiments_uncertainties = experiments.iloc[:,0:16]
experiments_uncertainties = experiments.drop(experiments.columns[16:-2], axis=1)
experiments_levers = experiments.iloc[:,16:-1]

In [ ]:
data_ordered = np.sort(data)

In [ ]:
plt.plot(data)

In [ ]:
plt.plot(data_ordered)

In [ ]:
len(np.unique(data))

In [ ]:
len(data)

In [ ]:
data = outcomes['Expected Annual Damage']
np.percentile(data, 10)

In [ ]:
np.percentile(data, 90)

In [ ]:
from ema_workbench.analysis import prim

In [ ]:
prim.Prim??

In [ ]:
data = outcomes['Expected Annual Damage']

y = data > np.percentile(data, 95)

prim_alg = prim.Prim(experiments_uncertainties,y, threshold=0.8)
box1 = prim_alg.find_box()

In [ ]:
box1.show_tradeoff()
plt.show()

In [ ]:
#box1.inspect(style='graph')
#plt.show()

In [ ]:
#box1.inspect(-1)

In [ ]:
#box1.inspect(25, style='graph')
#plt.show()

In [ ]:
from ema_workbench.analysis import dimensional_stacking
box1.select(-1)
box1.show_pairs_scatter()
fig = plt.gcf()
fig.set_size_inches(12,12)
plt.show()

In [ ]:
data = outcomes['Expected Annual Damage']

y = data > np.percentile(data, 95)

dimensional_stacking.create_pivot_plot(experiments_uncertainties, y,nbins=3, nr_levels=3)
#plt.savefig('Worst-ExpectedAnnualDamage-15000s-1p.png')
plt.show()

In [ ]:
data = outcomes['Total Investment Costs']

y = data > np.percentile(data, 90)

dimensional_stacking.create_pivot_plot(experiments_uncertainties, y,nbins=3, nr_levels=3)
#plt.savefig('Worst-TotalInvestmentCosts-15000s-1p.png')
plt.show()

In [ ]:
data = outcomes['Expected Number of Deaths']

y = data > np.percentile(data, 99)

dimensional_stacking.create_pivot_plot(experiments_uncertainties, y,nbins=3, nr_levels=3)
#plt.savefig('Worst-ExpectedNumberOfDeaths-15000s-1p.png')
plt.show()


In [ ]:
list(experiments_uncertainties)

In [ ]:
box1.select(-1)
fig = box1.show_pairs_scatter()
fig = plt.gcf()
fig.set_size_inches(12,12)
plt.show()

In [ ]:
box2 = prim_alg.find_box()

In [ ]:
from ema_workbench.analysis import cart
cart_alg = cart.CART(experiments_uncertainties,y, 0.05)
cart_alg.build_tree()

In [ ]:
import pydotplus
import pydot

In [ ]:
cart_alg.show_tree()

In [ ]:
prim.show_boxes_individually(boxes, results, uv=[], filter=True)

In [ ]:
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)
 
with MultiprocessingEvaluator(model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=1000, policies=50)